<a href="https://colab.research.google.com/github/oliveiraprg/text-summarization-with-python/blob/main/sumarizacao_com_similaridade_do_cosseno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sumarização com similaridade do cosseno

## Preparação do ambiente

In [50]:
import re
import nltk
import string
from nltk.cluster.util import cosine_distance
import networkx as nx
import numpy as np
import spacy
from spacy import displacy

In [51]:
!python -m spacy download pt_core_news_sm

2022-10-06 23:57:09.537207: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 2.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [52]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
stop_words = nltk.corpus.stopwords.words('portuguese')

In [54]:
def pre_processamento(texto):
  texto_formatado = texto.lower()
  tokens = []
  for token in nltk.word_tokenize(texto_formatado):
    tokens.append(token)
  tokens = [palavra for palavra in tokens if palavra not in stop_words and palavra not in string.punctuation]
  texto_formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])
  return texto_formatado

In [55]:
texto_original = """A inteligência artificial é a inteligência similar à humana máquinas.
                    Definem como o estudo de agente artificial com inteligência.
                    Ciência e engenharia de produzir máquinas com inteligência.
                    Resolver problemas e possuir inteligência. 
                    Relacionada ao comportamento inteligente. 
                    Construção de máquinas para raciocinar. 
                    Aprender com os erros e acertos. 
                    Inteligência artificial é raciocinar nas situações do cotidiano."""

In [56]:
texto_original = re.sub(r'\s+', ' ', texto_original)

## Função para calcular similaridade entre sentenças

In [57]:
sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto_original)]
sentencas_formatadas = [pre_processamento(sentencas_original) for sentencas_original in sentencas_originais]

In [58]:
def calcula_similaridade_sentencas(primeira_sentenca, segunda_sentenca):
  primeira_palavras = [palavra for palavra in nltk.word_tokenize(primeira_sentenca)]
  segunda_palavras = [palavra for palavra in nltk.word_tokenize(segunda_sentenca)]
  todas_palavras = list(set(primeira_palavras + segunda_palavras))
  primeiro_vetor = [0] * len(todas_palavras)
  segundo_vetor = [0] * len(todas_palavras)

  for palavra in primeira_palavras:
    primeiro_vetor[todas_palavras.index(palavra)] += 1 

  for palavra in segunda_palavras:
    segundo_vetor[todas_palavras.index(palavra)] += 1 

  return 1 - cosine_distance(primeiro_vetor, segundo_vetor)

##Função para gerar a matriz de similaridade

In [59]:
def calcula_matriz_similaridade(sentencas):
  matriz_similaridade = np.zeros((len(sentencas), len(sentencas)))
  for linha in range(len(sentencas)):
    for coluna in range(len(sentencas)):
      if linha == coluna:continue
      matriz_similaridade[linha][coluna] = calcula_similaridade_sentencas(sentencas[linha], sentencas[coluna])
  return matriz_similaridade

## Função para sumarizar

In [60]:
def sumarizar(texto):
  sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
  sentencas_formatadas = [pre_processamento(sentenca_original) for sentenca_original in sentencas_originais]
  quantidade_sentencas = int(len(sentencas_originais) / 2.5)
  matriz_similaridade = calcula_matriz_similaridade(sentencas_formatadas)
  grafo_similaridade = nx.from_numpy_array(matriz_similaridade)
  notas = nx.pagerank(grafo_similaridade)
  notas_ordenadas = sorted(((notas[indice_nota], nota) for indice_nota, nota in enumerate(sentencas_originais)), reverse=True)
  melhores_sentencas = []
  for sentenca in range(quantidade_sentencas):
    melhores_sentencas.append(notas_ordenadas[sentenca][1])
  return sentencas_originais, melhores_sentencas, notas_ordenadas

In [61]:
sentencas_originais, melhores_sentencas, notas_ordenadas = sumarizar(texto_original)

In [62]:
sentencas_originais, melhores_sentencas, notas_ordenadas 

(['A inteligência artificial é a inteligência similar à humana máquinas.',
  'Definem como o estudo de agente artificial com inteligência.',
  'Ciência e engenharia de produzir máquinas com inteligência.',
  'Resolver problemas e possuir inteligência.',
  'Relacionada ao comportamento inteligente.',
  'Construção de máquinas para raciocinar.',
  'Aprender com os erros e acertos.',
  'Inteligência artificial é raciocinar nas situações do cotidiano.'],
 ['A inteligência artificial é a inteligência similar à humana máquinas.',
  'Inteligência artificial é raciocinar nas situações do cotidiano.',
  'Ciência e engenharia de produzir máquinas com inteligência.'],
 [(0.22820924040178003,
   'A inteligência artificial é a inteligência similar à humana máquinas.'),
  (0.1839190802151221,
   'Inteligência artificial é raciocinar nas situações do cotidiano.'),
  (0.1633191450783496,
   'Ciência e engenharia de produzir máquinas com inteligência.'),
  (0.1543717033327776,
   'Definem como o estudo

## Visualização do resumo

In [63]:
def visualiza_resumo(titulo, lista_sentencas, melhores_sentencas):
  from IPython.core.display import HTML
  texto = ''
  resumo = ''
  display(HTML(f'<h1>{titulo}</h1>'))
  display(HTML(f'<h2>Texto com resumo destacado - {titulo}</h2>'))
  for sentenca in lista_sentencas:
    if sentenca in melhores_sentencas:
      texto += str(sentenca).replace(sentenca, f'<mark>{sentenca} </mark>')
      resumo += str(sentenca).replace(sentenca, f'{sentenca} ')
    else:
      texto += f'{sentenca} '
  display(HTML(f'{texto}'))
  display(HTML(f'<h2>Resumo de - {titulo}</h2>'))
  display(HTML(f'{resumo}'))

## Extração de texto da internet

In [64]:
!pip install goose3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [65]:
from goose3 import Goose

In [66]:
g = Goose()
url = 'https://iaexpert.academy/2020/11/09/ia-preve-resultado-das-eleicoes-americanas/'
artigo = g.extract(url)

In [67]:
sentencas_originais, melhores_sentencas, notas_ordenadas = sumarizar(artigo.cleaned_text)

In [68]:
visualiza_resumo(artigo.title, sentencas_originais, melhores_sentencas)

## Lematização com algoritimo de Luhn

In [69]:
pln = spacy.load('pt_core_news_sm')

In [70]:
def pre_processamento_lematizacao(texto):
  texto = texto.lower()
  texto = re.sub("\s+", ' ', texto)
  documento = pln(texto)
  tokens = []
  for token in documento:
    tokens.append(token.lemma_)
  tokens = [palavra for palavra in tokens if palavra not in stop_words and palavra not in string.punctuation]
  texto_formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])
  return texto_formatado

In [71]:
def sumarizar_lematizacao(texto):
  sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
  sentencas_formatadas = [pre_processamento_lematizacao(sentenca_original) for sentenca_original in sentencas_originais]
  palavras = [palavra.lower() for sentenca in sentencas_formatadas for palavra in nltk.word_tokenize(sentenca)]
  quantidade_sentencas = int(len(sentencas_originais) / 2.5)
  matriz_similaridade = calcula_matriz_similaridade(sentencas_formatadas)
  grafo_similaridade = nx.from_numpy_array(matriz_similaridade)
  notas = nx.pagerank(grafo_similaridade)
  notas_ordenadas = sorted(((notas[indice_nota], nota) for indice_nota, nota in enumerate(sentencas_originais)), reverse=True)
  melhores_sentencas = []
  for sentenca in range(quantidade_sentencas):
    melhores_sentencas.append(notas_ordenadas[sentenca][1])
  return sentencas_originais, melhores_sentencas, notas_ordenadas

In [72]:
sentencas_originais_lematizacao, melhores_sentencas_lematizacao, notas_ordenadas_lematizacao = sumarizar_lematizacao(artigo.cleaned_text)

In [73]:
visualiza_resumo(artigo.title, sentencas_originais_lematizacao, melhores_sentencas_lematizacao)